In [1]:
#import libraries and the 'get_connection' function from env
import pandas as pd
import numpy as np
import env
import os
from env import get_connection
import seaborn as sns
import matplotlib.pyplot as plt

# Acquiring the data from the SQL server

In [2]:
def get_zillow():
    '''
    looking for an already existing zillow csv on the local machine
    '''
    if os.path.isfile('zillow.csv'):
        return pd.read_csv('zillow.csv')
    else:
        '''
        if there is no existing csv, then connect to the SQL server and get the information from 
        telco_churn db
        '''
        url = get_connection('zillow')
        '''
        use the query to rename columns too
        '''
        query = '''
                SELECT bedroomcnt as bedroom_count, bathroomcnt as bathroom_count, 
                calculatedfinishedsquarefeet as calc_finished_square_ft, 
                taxvaluedollarcnt as tax_value_dollar_count, yearbuilt as year_built, 
                taxamount as tax_amount, fips as fips_code 
                FROM properties_2017 
                JOIN propertylandusetype USING(propertylandusetypeid)
                WHERE propertylandusedesc = 'Single Family Residential'
                '''
        
        df = pd.read_sql(query, url)
        '''
        saving the newly queried SQL table to a csv so it
        can be used instead of connecting to the SQL server
        every time I want this info
        '''
        df.to_csv('zillow.csv', index=False)
        return df

In [3]:
df= get_zillow()

In [4]:
#Taking a look at the data
df.head()

,bedroom_count,bathroom_count,calc_finished_square_ft,tax_value_dollar_count,year_built,tax_amount,fips_code
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [5]:
df.shape

(2152863, 7)

# Identify and handle null values

In [6]:
#check for null values in each column

bec=df['bedroom_count'].isnull().sum()
bac=df['bathroom_count'].isnull().sum()
cfsf=df['calc_finished_square_ft'].isnull().sum()
tcdc=df['tax_value_dollar_count'].isnull().sum()
yb=df['year_built'].isnull().sum()
ta=df['tax_amount'].isnull().sum()
fc=df['fips_code'].isnull().sum()

print(f'number of bedroom count nulls is: {bec}')
print(f'number of bathroom count nulls is: {bac}')
print(f'number of calculated finished square feet nulls is: {cfsf}')
print(f'number of tax value dollar count nulls is: {tcdc}')
print(f'number of year built nulls is: {yb}')
print(f'number of tax amount nulls is: {ta}')
print(f'number of fips code nulls is: {fc}')

number of bedroom count nulls is: 11
number of bathroom count nulls is: 11
number of calculated finished square feet nulls is: 8484
number of tax value dollar count nulls is: 493
number of year built nulls is: 9337
number of tax amount nulls is: 4442
number of fips code nulls is: 0


### Null values are only a small part of the dataset so I am going to drop those rows

In [7]:
#Dropping nulls
df=df.dropna()

#Checking null values again
bec=df['bedroom_count'].isnull().sum()
bac=df['bathroom_count'].isnull().sum()
cfsf=df['calc_finished_square_ft'].isnull().sum()
tcdc=df['tax_value_dollar_count'].isnull().sum()
yb=df['year_built'].isnull().sum()
ta=df['tax_amount'].isnull().sum()
fc=df['fips_code'].isnull().sum()

print(f'number of bedroom count nulls is: {bec}')
print(f'number of bathroom count nulls is: {bac}')
print(f'number of calculated finished square feet nulls is: {cfsf}')
print(f'number of tax value dollar count nulls is: {tcdc}')
print(f'number of year built nulls is: {yb}')
print(f'number of tax amount nulls is: {ta}')
print(f'number of fips code nulls is: {fc}')

number of bedroom count nulls is: 0
number of bathroom count nulls is: 0
number of calculated finished square feet nulls is: 0
number of tax value dollar count nulls is: 0
number of year built nulls is: 0
number of tax amount nulls is: 0
number of fips code nulls is: 0


### Check data types and make sure they are the correct type for the kind of feature

In [8]:
df.dtypes

bedroom_count              float64
bathroom_count             float64
calc_finished_square_ft    float64
tax_value_dollar_count     float64
year_built                 float64
tax_amount                 float64
fips_code                  float64
dtype: object

**All features are floats so there is no datatype change necessary**

# Create wrangle_zillow function

In [15]:
def wrangle_zillow():
    '''
    looking for an already existing zillow csv on the local machine
    '''
    if os.path.isfile('zillow.csv'):
        return pd.read_csv('zillow.csv')
    else:
        '''
        if there is no existing csv, then connect to the SQL server and get the information from 
        telco_churn db
        '''
        url = get_connection('zillow')
        '''
        use the query to rename columns too
        '''
        query = '''
                SELECT bedroomcnt as bedroom_count, bathroomcnt as bathroom_count, 
                calculatedfinishedsquarefeet as calc_finished_square_ft, 
                taxvaluedollarcnt as tax_value_dollar_count, yearbuilt as year_built, 
                taxamount as tax_amount, fips as fips_code 
                FROM properties_2017 
                JOIN propertylandusetype USING(propertylandusetypeid)
                WHERE propertylandusedesc = 'Single Family Residential'
                '''
        
        df = pd.read_sql(query, url)
        '''
        drop null values
        '''
        df=df.dropna()
        '''
        saving the newly queried SQL table to a csv so it
        can be used instead of connecting to the SQL server
        every time I want this info
        '''
        df.to_csv('zillow.csv', index=False)
        return df

In [19]:
zillow= wrangle_zillow()

In [20]:
zillow.isnull().sum()

bedroom_count              0
bathroom_count             0
calc_finished_square_ft    0
tax_value_dollar_count     0
year_built                 0
tax_amount                 0
fips_code                  0
dtype: int64

In [21]:
zillow.shape

(2140235, 7)

# Move wrangle_zillow function to .py